In [1]:
"""
Calculate grid cell aspen patch metrics from a TreeMap-based aspen mask
Author: maxwell.cook@colorado.edu
"""

import os, sys
import geopandas as gpd

os.environ["PROJ_LIB"] = "/opt/miniconda3/envs/aspen-fire/share/proj"

# Custom functions
sys.path.append(os.path.join(os.getcwd(),'code/'))
from __functions import *

proj = 'EPSG:5070'

maindir = '/Users/max/Library/CloudStorage/OneDrive-Personal/mcook/'
projdir = os.path.join(maindir, 'aspen-fire/Aim2/')

print("Ready to go !")

Ready to go !


In [2]:
# load the grid statistics
fp = os.path.join(projdir,'data/tabular/mod/gridstats_tm_lf.csv')
gs = pd.read_csv(fp)
# filter to only grid cells with any aspen
gs_aspen = gs[gs['species_gp'] == 'quaking aspen']
print(f"\nThere are {len(gs_aspen['grid_idx'].unique())} grid cells with any aspen.\n")


There are 28761 grid cells with any aspen.



In [3]:
# load the spatial grids, subset to grid stats data frame
fp = os.path.join(projdir,'data/spatial/mod/VIIRS/viirs_snpp_jpss1_afd_latlon_fires_pixar_gridstats.gpkg')
grids = gpd.read_file(fp)
# create a unique ID
grids['grid_idx'] = grids['Fire_ID'].astype(str) + grids['grid_index'].astype(str)
grids['grid_idx'] = grids['grid_idx'].astype(int)
# subset the spatial grids
grids = grids[grids['grid_idx'].isin(gs_aspen['grid_idx'].unique())]
print(f"\nThere are [{len(grids)}] grids across [{len(grids['Fire_ID'].unique())}] fires.\n")


There are [28761] grids across [71] fires.



In [4]:
# load the aspen mask
fp = os.path.join(projdir, "data/spatial/mod/USFS/tm_aspen_mask.tif")
# Load with rioxarray (handles CRS, nodata, etc.)
aspen_da = rxr.open_rasterio(fp, chunks=True).squeeze()
print(aspen_da.dtype)
print(aspen_da.rio.nodata)
print(aspen_da.rio.transform())

uint8
255
| 30.00, 0.00,-1188315.00|
| 0.00,-30.00, 2248365.00|
| 0.00, 0.00, 1.00|


In [6]:
# run it
shape = aspen_da.shape
transform = aspen_da.rio.transform()

results = []
for row in tqdm(grids.itertuples(index=False), total=len(grids)):
    idx = row.grid_idx # grid index
    geom = row.geometry # geometry for creating window

    try:
        # gather the grid geometry as a window
        row_start, row_stop, col_start, col_stop = geometry_to_window(geom, transform, shape)

        # slice the array using grid geometry
        arr = aspen_da.isel(
            y=slice(row_start, row_stop),
            x=slice(col_start, col_stop)
        ).values.squeeze()

        # get the window size in ha (gut check)
        n_pixels = arr.shape[0] * arr.shape[1]
        pixel_area_ha = (30 * 30) / 10_000  # 0.09 ha
        window_area_ha = n_pixels * pixel_area_ha

        # make sure there is valid data
        if np.count_nonzero(arr == 1) == 0:
            continue

        # compute the region metrics
        metrics = compute_patch_metrics(arr)  # ← scikit-image function
        if metrics:
            metrics['grid_idx'] = idx
            metrics['grid_ha'] = window_area_ha
            results.append(metrics)

    except Exception as e:
        print(f"Grid {idx} failed: {e}")
        continue

# gather the results
df_patch = pd.DataFrame(results)
df_patch.head()

,n_patch,mean_patch,large_patch,total_aspen_ha,grid_idx,grid_ha
0,6,0.135000,0.27,0.81,161955422,14.04
1,4,0.247500,0.45,0.99,161957669,14.04
2,16,0.275625,0.99,4.41,161957670,15.21
3,4,0.157500,0.27,0.63,161957671,14.04
4,7,0.154286,0.27,1.08,161957672,15.21


In [7]:
df_patch['aspen_pr'] = df_patch['total_aspen_ha'] / df_patch['grid_ha'] * 100
df_patch.head()

,n_patch,mean_patch,large_patch,total_aspen_ha,grid_idx,grid_ha,aspen_pr
0,6,0.135000,0.27,0.81,161955422,14.04,5.769231
1,4,0.247500,0.45,0.99,161957669,14.04,7.051282
2,16,0.275625,0.99,4.41,161957670,15.21,28.994083
3,4,0.157500,0.27,0.63,161957671,14.04,4.487179
4,7,0.154286,0.27,1.08,161957672,15.21,7.100592


In [8]:
# save the file out.
out_fp = os.path.join(projdir,'data/tabular/mod/gridstats_aspen_patches.csv')
df_patch.to_csv(out_fp)
print(f"Saved file to: {out_fp}")

Saved file to: /Users/max/Library/CloudStorage/OneDrive-Personal/mcook/aspen-fire/Aim2/data/tabular/mod/gridstats_aspen_patches.csv
